In [2]:
import io
import os
# import telebot
import random
from PIL import Image, ImageDraw , ImageFont

In [8]:
random.seed(444)
board = Image.open('img/b.jpg')
survived = Image.open('img/survived.jpeg')

In [9]:
def sizing(img: Image, povtorka:bool):
    img_width, img_height = img.size
    if povtorka:
        b_width, b_height = board.size
        return img_width/ (b_width/2) , img_height/ (b_height/2)
    else:
        s_width, s_height = survived.size
        return img_width/ (s_width/2) , img_height/ (s_height/1.5)

def verdict(img: Image, povtorka:bool):
    if povtorka:
        width, height = img.size
        k1,k2 = sizing(img,povtorka)
        img = img.resize((int(width/k1), int(height/k2)))
        board.paste(img,(125,150))
        return board
    else:
        width, height = img.size 
        k1,k2 = sizing(img,povtorka)
        img = img.resize((int(width/k1), int(height/k2)))
        survived.paste(img,(200,130))
        return survived

In [4]:
def pillow_to_bytes(pillow: Image):
    byte = io.BytesIO()
    pillow.save(byte, format=pillow.format)
    byte = byte.getvalue()
    return byte

def bytes_to_pillow(byte: bytes):
    pillow = Image.open(io.BytesIO(byte))
    return pillow

In [6]:
downloaded_file = pillow_to_bytes(Image.open("dataset2/povtorka/real_00001.jpg"))
type(downloaded_file)

bytes

In [10]:
res = pillow_to_bytes(verdict(bytes_to_pillow(downloaded_file), random.randint(0, 1)))
type(res)

bytes